Robert Miller

In [ ]:
from essentia.standard import *
import glob

In [ ]:
myLib = glob.glob('<some directory here>/*.wav')

In [ ]:
sndFiles = []
for snd in myLib:
    loader = MonoLoader(filename=snd)
    sndFiles.append(loader())

In [ ]:
w = Windowing(type='hann')
spectrum = Spectrum()
pitchYin = PitchYinFFT()

In [ ]:
titles = []
for ml in myLib:
    titles.append(ml[55:])

### Pitch Histograms

In [ ]:
def getPitches(audio, windowSize=2048, hop=1024):
    pitches = []
    confidences = []
    for frame in FrameGenerator(audio, frameSize=windowSize, hopSize=hop):
        pitch, pConf = pitchYin(spectrum(w(frame)))
        pitches.append(pitch)
        confidences.append(pConf)
    return pitches, confidences

In [ ]:
p1, c1 = getPitches(sndFiles[0])

In [ ]:
plot(p1)

In [ ]:
plot(p1[2000:2100])
twinx()
plot(c1[2000:2100],'r')

In [ ]:
pitches = []
confidences = []
for snd in sndFiles:
    p,c = getPitches(snd)
    pitches.append(p)
    confidences.append(c)

In [ ]:
#Code from Class
def midi2Hz(midinote, tuning=440.0):
    return tuning * (2**((midinote - 69)/12.0))

num_freqs = 8*12 # eight octaves from C0
quant_freqs = [midi2Hz(i + 24) for i in range(num_freqs)]

def quantize_freq(freq_list, quant_freqs, quant_offset=24):
    quantized = zeros_like(freq_list)
    for i in range(len(freq_list)):
        arg = argwhere(array(quant_freqs) > freq_list[i])
        if arg.size == 0 or arg[0] == 0:
            quantized[i] = 0
        elif quant_freqs[arg[0]] - freq_list[i] > freq_list[i] - quant_freqs[arg[0] - 1]:
            quantized[i] = arg[0] - 1
        else:
            quantized[i] = arg[0]
    return quantized + quant_offset

In [ ]:
quantized_pitches = []
for p in pitches:
    quantized_pitches.append(quantize_freq(p,quant_freqs))

In [ ]:
hist(quantized_pitches[0],arange(40,100));
title(titles[0])

In [ ]:
hist(quantized_pitches[1],arange(40,100));
title(titles[1])

In [ ]:
hist(quantized_pitches[2],arange(40,100));
title(titles[2])

In [ ]:
hist(quantized_pitches[3],arange(40,100));
title(titles[3])

In [ ]:
hist(quantized_pitches[4],arange(40,100));
title(titles[4])

### Self-Similarity Matrix

In [ ]:
winSize = 200
hopSize = 100
signal = pitches[1]
win_start = arange(0,len(signal)-winSize,hopSize)
ssm = []
for starti in win_start:
    for startj in win_start:
        wini = signal[starti:starti+winSize]
        winj = signal[startj:startj+winSize]
        ssm.append(corrcoef(wini,winj)[0,1])

In [ ]:
ssm = array(ssm).reshape(len(win_start), len(win_start))

In [ ]:
imshow(ssm,cmap='gray')
title(titles[1])

In [ ]:
def constructSSM(sig):
    winSize = 200
    hopSize = 100
    signal = sig
    win_start = arange(0,len(signal)-winSize,hopSize)
    ssm = []
    for starti in win_start:
        for startj in win_start:
            wini = signal[starti:starti+winSize]
            winj = signal[startj:startj+winSize]
            ssm.append(corrcoef(wini,winj)[0,1])
    ssm = array(ssm).reshape(len(win_start), len(win_start))
    return ssm

In [ ]:
ssms = []
for p in pitches:
    ssms.append(constructSSM(p))

In [ ]:
imshow(ssms[0])
title(titles[0])

In [ ]:
imshow(ssms[2])
title(titles[2])

In [ ]:
imshow(ssms[3],cmap='gray')
title(titles[3])

In [ ]:
imshow(ssms[4])

In [ ]:
imshow(ssms[4],cmap='gray')
title(myLib[4][55:])